# TP 1 et 2 : Accès aux données avec index
# sujet

date de modification : 18/01/2024

BINOME

NOM 1: TAFOUGHALT

Prénom 1: Anyes

NOM 2: DJEGHALI

Prénom 2: Racha Nadine

TP à rendre : **REDIGER des explications détaillées et argumentées** pour les solutions que vous proposez




Objectifs:
Savoir organiser des données en pages pour permettre de modifier un tuple en ne modifiant qu'une seule page.

Comprendre les méthodes d'accès suivantes :

*   Lecture séquentielle d'une fichier : "table access full"
*   Lecture d'un tuple dont on connait le rowid : "table access by index rowid"
*   Opération de sélection par lecture séquentielle et filtrage

Comprendre les méthodes d'indexation :

*   Créer un index
*   Opération de sélection par index et lecture par rowid

Mise à jour de données
*   Sélectionner un tuple et modifier un de ses attributs
*   Modifier l'index en conséquence lorsque l'attibut modifié est indexé

Persistence
*   Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement
*   Adapter en conséquence les opérations de modification de l'index


In [50]:
import os
import shutil as sh
import numpy as np
import pandas as pd
import random

# from sortedcontainers import SortedDict
import sortedcontainers

from string import ascii_lowercase
import time

# le nom de la table
TABLE = "T"
print("le nom de la table est", TABLE)


# le nom du fichier qui contient les données de la table
def nom_fichier(table):
    return table + ".csv"

le nom de la table est T


# Générer les données du TP

Création du fichier contenant un exemple de données.
Ce sont des données au format csv. On suppose que chaque ligne correspond à un tuple d'une table **T** ayant *n* attributs :

$$ T (a_0, a_1, a_2, ..., a_{n-1})$$

Le premier attribut $a_0$ est unique.

Les attributs $a_1$ à $a_{n-2}$ ne sont pas uniques : il y a en moyenne $2^k$ tuples par valeurs de $a_k$ soit 2 tuples par valeur de $a_1$ et 4 tuples par valeurs de $a_2$.

Les attributs sont des nombres entiers, multiples de 10, sauf le dernier qui est une chaine de caractères (on choisit des mutliples de 10 pour représenter le cas plus général de domaines contenant des valeurs non consécutives).



In [51]:
# dure environ 20s pour 2M lignes
# dure environ 40s pour 5M lignes


def genere_fichier(nb_lignes, nb_attributs, longueur_dernier_attribut, table):
  # attribut_chaine_caracteres = "".join(choice(ascii_lowercase) for i in range(longueur_dernier_attribut))
  attribut_chaine_caracteres = ''.join('-' for i in range(longueur_dernier_attribut))
  # print("le dernier attribut de chaque tuple est la chaine de caracètes :", attribut_chaine_caracteres)

  # reproductibilité des données générées
  rng = np.random.default_rng(seed=1)

  data={}

  # le premier attribut est unique.
  nb_valeurs_distinctes = nb_lignes
  data['a0'] = 10 * rng.permutation(np.arange(nb_valeurs_distinctes))

  # les attributs suivants ont des domaines plus petits :
  for i in range(1, nb_attributs):
    # on divise le domaine par 2 à chaque itération
    nb_valeurs_distinctes = max(2, int(nb_valeurs_distinctes / 2))
    data[f'a{i}'] = 10 * rng.integers(0, nb_valeurs_distinctes, nb_lignes)

  # on concatène "verticalement" les attributs dans un dataframe pour former des tuples sur lesquels on peut itérer.
  df = pd.DataFrame(data)
  # rmq: le dernier attribut est une chaine de caractères
  b = [str(e)[1:-1] + f",{attribut_chaine_caracteres}\n" for e in df.itertuples(index=False, name=None)]



  # on stocke les données dans un fichier
  fichier = nom_fichier(table)
  print("écriture des données dans le fichier", fichier)
  with open(fichier, "w") as f:
    # écriture groupée de tous les tuples
    f.write(''.join(b))



In [52]:
nb_lignes = 2 * 1000 * 1000
# nb_lignes = 5 * 1000 * 1000
nb_attributs = 7
longueur_dernier_attribut = 100

t1 = time.time()
genere_fichier(nb_lignes, nb_attributs, longueur_dernier_attribut, TABLE)
print(f"durée pour générer {nb_lignes} lignes :", round(time.time() - t1, 1), "s")

écriture des données dans le fichier T.csv
durée pour générer 2000000 lignes : 14.7 s


On affiche le début et la fin du fichier et son nombre de lignes ( = card(T))

In [53]:
%%bash -s "{TABLE}.csv"
echo "debut de $1 :"
head -n 2 $1
echo
echo "fin de $1 : "
tail -n 2 $1
echo
echo "nombre de lignes:"
wc -l $1


debut de T.csv :
19512870, 5751570, 1235270, 1344040, 1139220, 359300, 261790,----------------------------------------------------------------------------------------------------
3462320, 3047440, 4558650, 2392580, 332870, 445910, 35390,----------------------------------------------------------------------------------------------------

fin de T.csv : 
4418430, 9611540, 268500, 660850, 881140, 311230, 273070,----------------------------------------------------------------------------------------------------
1896950, 9749280, 4493840, 1757100, 1083240, 323240, 177530,----------------------------------------------------------------------------------------------------

nombre de lignes:
2000000 T.csv


# Lecture séquentielle

On définit un *iterateur* pour lire séquentiellement chaque ligne de la table stockée entièrement dans un seul fichier.
Le mot python *yield* permet de définir un itérateur qui est retourné par la fonction.

Cet itérateur est invoqué pour lire la table et appliquer un filtre.



In [54]:
def lecture_sequentielle(table):
  fichier = nom_fichier(table)
  with open(fichier, "r") as f:
    for i, line in enumerate(f):
      yield i, line

def filtrer_table(table, valeur_recherchee):
  for i, line in lecture_sequentielle(table):
      a = int(line.split(',')[0])
      if a == valeur_recherchee :
        print(f"ligne {i} :", line.strip())


nb_valeurs_distinctes = nb_lignes
s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_table(TABLE, s)
print("durée :", round(time.time() - t1, 3), "s")

valeur recherchée : 963852


durée : 2.713 s


# Découper une table en pages

On organise les données en pages.
Pour faciliter le TP, chaque page est représentée par un "petit" fichier mais en réalité une page est un bloc d'un fichier.

Dans la suite du TP, on accédera toujours aux pages.
Le fichier créé initialement, contenant tous les tuples, ne sera plus utilisé.

In [55]:
def page_dir_name(table):
  return table + "_pages"

def decoupe_table_en_pages(table, nb_tuple_par_page):
  page_dir = page_dir_name(table)

  # vider le dossier qui contiendra les pages
  if(os.path.exists(page_dir)):
    sh.rmtree(page_dir)
  os.makedirs(page_dir, exist_ok=True)

  # lire le fichier contenant tous les tuples
  p=0
  lines = []
  for i, line in lecture_sequentielle(table):
    lines.append(line)
    if (i+1) % nb_tuple_par_page == 0:

      # créer une page
      p += 1
      with open(page_dir + f"/page{p}", "w") as fp:
        fp.write(''.join(lines))
      lines = []

  # créer une dernière page, si nécessaire
  if len(lines) > 0:
    p +=1
    with open(page_dir + f"/page{p}", "w") as fp:
        fp.write(''.join(lines))

  print("nb pages créées :", p)


print("les pages sont stockées dans le dossier", page_dir_name(TABLE) )

decoupe_table_en_pages(TABLE, nb_tuple_par_page=1000)

les pages sont stockées dans le dossier T_pages


nb pages créées : 2000


Afficher (pour quelques pages) le nombre de tuples contenus dans une page

une solution en bash :

In [56]:
%%bash -s "$TABLE"
wc -l $1_pages/* | head -n 3

     1000 T_pages/page1
     1000 T_pages/page10
     1000 T_pages/page100


une autre solution en python :

In [57]:
page_dir = page_dir_name(TABLE)
l = os.listdir(page_dir)
random.seed(1)
for i in range(3):
  une_page = random.choice(l)
  with open(page_dir + f"/{une_page}", 'r') as fp:
    lines = len(fp.readlines())
    print(f"la page {une_page} contient {lines} lignes")

la page page1402 contient 1000 lignes
la page page854 contient 1000 lignes
la page page1522 contient 1000 lignes


# Lecture séquentielle d'une table découpée en pages

In [58]:
def lecture_sequentielle_par_page(table):
  page_dir = page_dir_name(table)
  nb_pages = len(os.listdir(page_dir))
  for p in range(1, nb_pages+1) :
    with open(page_dir + f"/page{p}", "r") as f:
      for i, line in enumerate(f):
        tuple_courant = line.strip().split(',')
        yield p, i, tuple_courant

def filtrer_table_par_pages(table, valeur_recherchee):
  for page, position, tuple_courant in lecture_sequentielle_par_page(table):
    attribut0 = int(tuple_courant[0])
    if attribut0 == valeur_recherchee :
      print(f"page {page}, ligne {position} :", tuple_courant)

search = np.random.randint(nb_lignes)
print("valeur recherchée :", search)

t1 = time.time()
filtrer_table_par_pages(TABLE, search)
print("durée :", round(time.time() - t1, 2), "s")

valeur recherchée : 1613801


durée : 4.99 s


# Lecture d'un tuple dans une page

Cette fonction retourne le tuple situé dans la page *num_page* et à la position *position*

In [59]:
def lecture_tuple(table, num_page, position):
  page_dir = page_dir_name(table)
  with open(page_dir + f"/page{num_page}", "r") as f:
    lines = f.readlines()
    return lines[position].strip()

t1 = time.time()
print(lecture_tuple(TABLE, 123, 456))
print("done in", round((time.time() - t1)*1000, 1), "ms")

17233030, 9546020, 4178470, 117430, 31540, 530090, 250650,----------------------------------------------------------------------------------------------------
done in 1.7 ms


# Exercice 1 : Créer un index

## Créer un index unique pour l'attribut $a_0$

On sait que $a_0$ est unique.
Une entrée de l'index associe une *clé* à une *valeur* :
*   La *clé* est la valeur du premier attribut.
*   La *valeur* est un **rowid** formé des informations (page, position)



In [60]:
def creation_index_unique(table):
  index = {}
  for page, position, tuple_courant in lecture_sequentielle_par_page(table):
    attribut0 = int(tuple_courant[0])
    index[attribut0] = (page,position)
  return sortedcontainers.SortedDict(index)

t1 = time.time()
INDEX_UNIQUE_a0 = creation_index_unique(TABLE)
print("durée ", round(time.time() - t1, 3), "s")

durée  10.178 s


In [61]:
# vérifier l'index
s = 10 * np.random.randint(nb_lignes)
print(s, INDEX_UNIQUE_a0[s])

9538850 (499, 702)


## Créer un index non unique pour l'attribut $a_i$

On donne un nom de table et le numéro $i$ de l'attribut $a_i$ de la table.

In [62]:
def creation_index(table, numero_attribut):
  index = {}
  for page, position, tuple_courant in lecture_sequentielle_par_page(table):
    attribut_val = int(tuple_courant[numero_attribut])
    if(attribut_val not in index):
      index[attribut_val]=[(page,position)]
    else:
      index[attribut_val].append((page,position))
  return sortedcontainers.SortedDict(index)


t1 = time.time()
INDEX_a2 = creation_index(TABLE, 2)
print("duree de création de l'index pour l'attribut a2:", round(time.time() - t1, 3), "s")

duree de création de l'index pour l'attribut a2: 12.631 s


In [63]:
# # vérifier l'index
s = 10 * np.random.randint(nb_valeurs_distinctes/4)
print("valeur recherchée :", s)
for r in INDEX_a2[s]:
  print(r)

valeur recherchée : 4176740
(240, 847)
(547, 483)
(1078, 662)
(1209, 500)
(1821, 498)


# Exerccie 2 : Accès par index

## Accès ciblé

On veut retrouver les tuples telq qu'un attribut indexé a une valeur donnée.

###Index unique scan.
On recherche un unique tuple dont l'attribut indexé a une valeur donnée (car l'attribut est unique)

In [64]:
def acces_par_index_unique(index_unique, table, valeur_recherchee):
 num_page, position =  index_unique[valeur_recherchee]
 return lecture_tuple(table, num_page, position)

s = 10 * np.random.randint(nb_lignes)
print("valeur recherchée :", s)

t1 = time.time()
tuple = acces_par_index_unique(INDEX_UNIQUE_a0, TABLE, s)
print("resulat:", tuple)
print("done in", round((time.time() - t1)*1000, 2), "ms")

valeur recherchée : 9224270
resulat: 9224270, 8819320, 2908210, 171370, 882390, 263060, 223710,----------------------------------------------------------------------------------------------------
done in 2.61 ms


###Index scan
Accès pour rechercher les tuples dont l'attribut indexé a une valeur donnée. On suppose que l'attribut n'est pas unique et que plusieurs tuples sont retrouvés

In [65]:
def acces_par_index(index, table, valeur_recherchee):

    tuples_indices=  index[valeur_recherchee]
    tuples = []
    for page , position  in tuples_indices :
       tuples.append(lecture_tuple(table, page, position))
    return tuples



s = 10* np.random.randint(nb_lignes/4)
print("valeur recherchée :", s)

t1 = time.time()
for t in acces_par_index(INDEX_a2, TABLE, s):
  print(t)
print("done in", round((time.time() - t1)*1000, 2), "ms")

valeur recherchée : 1344280
43670, 2944140, 1344280, 2414210, 256370, 618290, 190770,----------------------------------------------------------------------------------------------------
17961130, 5800270, 1344280, 430700, 766790, 129870, 123260,----------------------------------------------------------------------------------------------------
18071160, 1507990, 1344280, 2218530, 587740, 173650, 16140,----------------------------------------------------------------------------------------------------
14560010, 6120300, 1344280, 363450, 891020, 469650, 240210,----------------------------------------------------------------------------------------------------
done in 7.04 ms


## Accès par intervalle
Index range scan



### Accès par intervalle sur un attribut unique
Accès pour rechercher les tuples dont l'attribut indexé est unique et a une valeur comprise dans un intervalle donné.
Indications, votre solution doit prendre en compte les exigences suivantes :
*  Les valeurs recherchées ne sont pas connues à l'avance. On sait seulement qu'elles sont incluses dans un intervalle. Ne pas supposer qu'on recherche des entiers consécutifs.
*  Les bornes de l'intervalle ne sont pas parmi les valeurs existantes de l'attribut. Par exemple, on peut rechercher les valeurs de $a_0$ comprises dans l'intervalle  [23 , 45].



In [66]:
# Exemple pour retrouver la première entrée de l'intervalle [23,45]

indice = INDEX_UNIQUE_a0.bisect_left(23)
print("l'indice de la première clé à retrouver est :", indice)
cle = INDEX_UNIQUE_a0.keys()[indice]
print("la première clé retrouvée est:", cle)
print("la valeur à retrouver est :",  INDEX_UNIQUE_a0[cle])

l'indice de la première clé à retrouver est : 3
la première clé retrouvée est: 30
la valeur à retrouver est : (1783, 941)


In [67]:
def acces_intervalle_par_index_unique(index_unique, table, borne_inf, borne_sup):
    indice_inf = index_unique.bisect_left(borne_inf)
    i = indice_inf
    tuples = []
    cle = index_unique.keys()[i]
    while( cle <=  borne_sup ):
      tuples.append(acces_par_index_unique(index_unique, table, cle))
      i += 1
      cle = index_unique.keys()[i]

    return tuples




s = 10 * np.random.randint(nb_valeurs_distinctes/4)
print("valeurs recherchées seront dans l'intervalle : [", s+3 , ",",s+23,"]")

t1 = time.time()
for t in acces_intervalle_par_index_unique(INDEX_UNIQUE_a0, TABLE, s + 3, s + 23):
  print(t)
print("done in", round(time.time() - t1, 2), "s" )

valeurs recherchées seront dans l'intervalle : [ 4721033 , 4721053 ]
4721040, 6932710, 1952880, 2312330, 612760, 491600, 97530,----------------------------------------------------------------------------------------------------
4721050, 8586970, 3578540, 1810690, 493410, 431490, 255740,----------------------------------------------------------------------------------------------------
done in 0.01 s


### Accès par intervalle sur un attribut NON unique
Accès pour rechercher les tuples dont l'attribut indexé n'est **pas** unique et a une valeur comprise dans un intervalle donné.
Votre solution doit prendre en compte les mêmes exigences que dans la question précédente.

In [68]:
def acces_intervalle_par_index(index, table, borne_inf, borne_sup):
    indice_inf = index.bisect_left(borne_inf)
    i = indice_inf
    tuples = []
    cle = index.keys()[i]
    while( cle <=  borne_sup ):
      tuples.extend(acces_par_index(index, table, cle))
      i += 1
      cle = index.keys()[i]

    return tuples

s = 10 * np.random.randint(nb_valeurs_distinctes / 4)
print("valeurs recherchées seront dans l'intervalle : [", s+3 , ",",s+33,"]")

t1 = time.time()

for t in acces_intervalle_par_index(INDEX_a2, TABLE, s + 3, s + 33):
  print(t)
print("done in", round(time.time() - t1, 2), "s")

valeurs recherchées seront dans l'intervalle : [ 2513263 , 2513293 ]
923330, 1155930, 2513270, 2486250, 603540, 587060, 65960,----------------------------------------------------------------------------------------------------
18295370, 5980220, 2513270, 633160, 1198720, 425810, 140600,----------------------------------------------------------------------------------------------------
11891130, 8744070, 2513270, 1578560, 716620, 221950, 267190,----------------------------------------------------------------------------------------------------
51570, 8081260, 2513270, 1448800, 898140, 118680, 13240,----------------------------------------------------------------------------------------------------
12240670, 5827940, 2513280, 1093360, 438580, 396170, 296770,----------------------------------------------------------------------------------------------------
5813610, 4234760, 2513280, 2347770, 124240, 141400, 151000,--------------------------------------------------------------------------

# Exercice 3 : Mise à jour de données




## Modifier la valeur d'un attribut d'un ou plusieurs tuples

Cela correspond à l'insctruction UPDATE table SET ... WHERE ...



### Modification d'un seul tuple

On donne une valeur *v* de l'attribut clé $a_0$. Ajouter 10 à l'attribut $a_1$. Cela correspond à l'instruction

update T
set a1 = a1+10
where a0 = *v*

Après la modification, accéder aux données pour vérifier que le tuple a bien été modifié. Par exemple, invoquer la fonction
acces_par_index_unique(index, table, v)



In [69]:
def ecriture_tuple(table, num_page, position,valeur_a_additionner,indince_a_modifier):
    page_dir = page_dir_name(table)
    with open(page_dir + f"/page{num_page}", "r+") as f:
        lines = f.readlines()
        a0 = int(lines[position].strip().split(',')[indince_a_modifier])
        print("Valeur initiale : ",a0)
        a0 += valeur_a_additionner
        print("Valeur apres modification : ",a0)
        l = lines[position]
        l = l.strip().split(',')
        l[indince_a_modifier] = " "+str(a0)
        lines[position] =','.join(l)+"\n"
        f.seek(0)
        f.writelines(lines)

def update_unique(index, table, v):
    num_page , position = index[v]
    ecriture_tuple(table,num_page,position,10,1)

In [70]:
# Test :
v = 18709620
print("avant la mise à jour " , acces_par_index_unique(INDEX_UNIQUE_a0,TABLE,v))
t1 = time.time()
update_unique(INDEX_UNIQUE_a0, TABLE, v)
t2 = round(time.time() - t1, 2)
print("aprés la mise à jour ", acces_par_index_unique(INDEX_UNIQUE_a0,TABLE,v))
print("done in", t2, "s")


avant la mise à jour  18709620, 5479280, 4390800, 2477770, 941380, 286730, 209680,----------------------------------------------------------------------------------------------------
Valeur initiale :  5479280
Valeur apres modification :  5479290
aprés la mise à jour  18709620, 5479290, 4390800, 2477770, 941380, 286730, 209680,----------------------------------------------------------------------------------------------------
done in 0.01 s


### Modification de plusieurs tuples

On donne une valeur *v* de l'attribut $a_2$ qui n'est pas unique. Ajouter 1 à l'attribut $a_3$ de tous les tuples pour lesquels $a_2 = v$

update T set a3 = a3+1 where a2=v



In [71]:
def update_plusieurs(index, table, v):
    tuples_indices=  index[v]
    dict_page_pos = {}
    for page , position  in tuples_indices :
        if(page in dict_page_pos):
            dict_page_pos[page].append(position)
        else :
            dict_page_pos[page] = [position]
    for num_page in list(dict_page_pos.keys()):
        page_dir = page_dir_name(table)
        with open(page_dir + f"/page{num_page}", "r+") as f:
            lines = f.readlines()
            for position in dict_page_pos[num_page] :
                print("page : ",num_page,", position : ",position)
                a3 = int(lines[position].strip().split(',')[3])
                print("Valeur initiale : ",a3)
                a3 += 10
                print("Valeur apres modification : ",a3)
                l = lines[position]
                l = l.strip().split(',')
                l[3] = " "+str(a3)
                lines[position] =','.join(l)+"\n"
            f.seek(0)
            f.writelines(lines)

In [72]:
v = 321940
print("avant la mise à jour ", acces_par_index(INDEX_a2,TABLE,v) )
t1 = time.time()
update_plusieurs(INDEX_a2, TABLE, v)
t2 = round(time.time() - t1, 2)
print("aprés la mise à jour " , acces_par_index(INDEX_a2,TABLE,v))
print("\nDone in", t2, "s")

avant la mise à jour  ['3907520, 9680, 321940, 298970, 436620, 418330, 191630,----------------------------------------------------------------------------------------------------', '1471360, 2647260, 321940, 1628930, 1247700, 587260, 186440,----------------------------------------------------------------------------------------------------', '14074930, 4504850, 321940, 817210, 747080, 268220, 142460,----------------------------------------------------------------------------------------------------', '7460450, 6149070, 321940, 813120, 1193110, 332950, 71640,----------------------------------------------------------------------------------------------------']
page :  854 , position :  869
Valeur initiale :  298970
Valeur apres modification :  298980
page :  973 , position :  359
Valeur initiale :  1628930
Valeur apres modification :  1628940
page :  1525 , position :  82
Valeur initiale :  817210
Valeur apres modification :  817220
page :  1866 , position :  364
Valeur initiale :  81312

##Modifier l'index en conséquence lorsque l'attribut modifié est indexé
Comerncer par créer un index sur l'attribut $a_3$

L'attribut $a_3$ étant maintenant indexé, la mise à jour de la question précédente implique d'actualiser l'index sur $a_3$ pour que les rowid des tuples qui contenaient l'ancienne valeur de $a_3$ soient associés à la nouvelle valeur de $a_3$.

In [73]:
def update_plusieurs_2(index2, index3, table, v):
    tuples_indices=  index2[v]
    dict_page_pos = {}
    for page , position  in tuples_indices :
        if(page in dict_page_pos):
            dict_page_pos[page].append(position)
        else :
            dict_page_pos[page] = [position]
    for num_page in list(dict_page_pos.keys()):
        page_dir = page_dir_name(table)
        with open(page_dir + f"/page{num_page}", "r+") as f:
            lines = f.readlines()
            for position in dict_page_pos[num_page] :
                print("page : ",num_page,", position : ",position)
                a3 = int(lines[position].strip().split(',')[3])
                index3[a3].remove((num_page,position))
                print("Valeur initiale : ",a3)
                a3 += 10
                print("Valeur apres modification : ",a3)
                l = lines[position]
                l = l.strip().split(',')
                l[3] = " "+str(a3)
                if(a3 not in index3):
                    index3[a3]=[(num_page,position)]
                else:
                     index3[a3].append((num_page,position))
                lines[position] =','.join(l)+"\n"
            f.seek(0)
            f.writelines(lines)

In [74]:
# Creation d'un index pour l'atribut a3 :
t1 = time.time()
INDEX_a3 = creation_index(TABLE, 3)
print("duree de création de l'index pour l'attribut a3:", round(time.time() - t1, 3), "s")

duree de création de l'index pour l'attribut a3: 10.568 s


In [75]:
#test :
v = 3468200
print("avant la mise à jour ", acces_par_index(INDEX_a2,TABLE,v) )
t1 = time.time()
update_plusieurs_2(INDEX_a2,INDEX_a3, TABLE, v)
t2 = round(time.time() - t1, 2)
print("aprés la mise à jour " , acces_par_index(INDEX_a2,TABLE,v))
print("\nDone in", t2, "s")

avant la mise à jour  ['10379420, 9830890, 3468200, 2393910, 454100, 231470, 116810,----------------------------------------------------------------------------------------------------', '8695360, 4248230, 3468200, 1700890, 103740, 207030, 72370,----------------------------------------------------------------------------------------------------']
page :  2 , position :  0
Valeur initiale :  2393910
Valeur apres modification :  2393920
page :  49 , position :  485
Valeur initiale :  1700890
Valeur apres modification :  1700900
aprés la mise à jour  ['10379420, 9830890, 3468200, 2393920, 454100, 231470, 116810,----------------------------------------------------------------------------------------------------', '8695360, 4248230, 3468200, 1700900, 103740, 207030, 72370,----------------------------------------------------------------------------------------------------']

Done in 0.01 s


# Exercice 4 : Persistence

Dans cette partie, on veut rendre les index persistents en stockant les entrées triées dans des pages. Cela permet d'utiliser les index plus efficacement en réduisant la durée pour les reconstruire.

## Stockage d'un index unique

Proposez une solution pour stocker les entrées **triées** d'un index dans plusieurs pages avec une taille de page fixée (10 000 rowids par page).
Etudier le cas d'un index unique et celui d'un index non unique

In [76]:
def index_unique_dir_name(table):
  return table + "_index_unique"
def index_unique(index , table , nb_index_par_page=10000):
    dir = index_unique_dir_name(table)

    if(os.path.exists(dir)):
        sh.rmtree(dir)
    os.makedirs(dir, exist_ok=True)

    index_keys = sorted(index.keys())
    p=0
    lines = []
    for i,key in enumerate(index_keys):
       lines.append(str(key)+","+str(index[key][0])+","+str(index[key][1])+"\n")
       if (i+1) % nb_index_par_page == 0:
            # créer une page
            p += 1
            with open( dir + f"/page{p}", "w") as fp:
                fp.write(''.join(lines))
            lines = []
    # créer une dernière page, si nécessaire
    if len(lines) > 0:
        p +=1
        with open(dir + f"/page{p}", "w") as fp:
            fp.write(''.join(lines))
    print("nb pages créées :", p)


       


In [77]:
#test
t1 = time.time()
index_unique(INDEX_UNIQUE_a0 , TABLE )
t2 = round(time.time() - t1, 2)
print("\nDone in", t2, "s")

nb pages créées : 200

Done in 8.64 s


In [78]:
def index_non_unique_dir_name(table,att):
  return table + "_index_non_unique_a"+str(att)
def index_non_unique(index , att, table , nb_index_par_page=10000):
    dir = index_non_unique_dir_name(table, att)

    if(os.path.exists(dir)):
        sh.rmtree(dir)
    os.makedirs(dir, exist_ok=True)

    index_keys = sorted(index.keys())
    p=0
    lines = []
    for key in index_keys:
        if len(lines) + len(index[key]) > nb_index_par_page :
            p += 1
            with open( dir + f"/page{p}", "w") as fp:
                fp.write(''.join(lines))
            lines = []
        for num_page, position in index[key]:
            lines.append(str(key)+","+str(num_page)+","+str(position)+"\n")
    # créer une dernière page, si nécessaire
    if len(lines) > 0 :
        p +=1
        with open(dir + f"/page{p}", "w") as fp:
            fp.write(''.join(lines))
    print("nb pages créées :", p)



In [79]:
#test

t1 = time.time()
INDEX_a1 = creation_index(TABLE, 1)
print("duree de création de l'index pour l'attribut a1:", round(time.time() - t1, 3), "s")
t1 = time.time()
index_non_unique(INDEX_a1 , 1, TABLE )
t2 = round(time.time() - t1, 2)
print("\nDone in", t2, "s")

duree de création de l'index pour l'attribut a1: 15.081 s
nb pages créées : 201

Done in 6.1 s


In [80]:
#test
t1 = time.time()
index_non_unique(INDEX_a2 , 2, TABLE )
t2 = round(time.time() - t1, 2)
print("\nDone in", t2, "s")

nb pages créées : 201

Done in 4.81 s


In [81]:
#test
t1 = time.time()
index_non_unique(INDEX_a3 , 3, TABLE )
t2 = round(time.time() - t1, 2)
print("\nDone in", t2, "s")

nb pages créées : 201

Done in 4.92 s


Montrez que vous pouvez reconstruire les index à partir des entrées stockées dans des pages.


In [82]:
#Reconstruire l'index unique
def reconstruction_index (index_dir):
  index = {}
  nb_pages = len(os.listdir(index_dir))
  for page in range(1,nb_pages+1):
    with open(index_dir +f"/page{page}", "r") as f:
      line = f.readline()
      cle , _ , _ = line.strip().split(',')
      if int(cle) not in index : 
        index[int(cle)] = page

  return sortedcontainers.SortedDict(index)

In [83]:
index_unique_non_dense = reconstruction_index ("T_index_unique")
index_a2_non_dense = reconstruction_index ("T_index_non_unique_a2")
index_a1_non_dense = reconstruction_index ("T_index_non_unique_a1")
index_a3_non_dense= reconstruction_index ("T_index_non_unique_a3")

## Adapter en conséquence les opérations de modification de l'index

Illustrer le cas :

update T set a3 = a3+0.5 where a1=v

où la nouvelle valeur $a_3' = a_3 + 0.5$ n'est pas déjà présente dans l'index. Il faut donc insérer une nouvelle clé  dans l'index de l'attribut $a_3$.
On suppose qu'il reste de la place dans une page de l'index pour insérer la nouvelle entrée (on peut avoir jusqu'à 12 000 rowids par page d'index).

In [84]:

def update_table_index(table, index , num_att , value_att , new_value_att , num_page , position) :
    index_dir = index_non_unique_dir_name(table, num_att)    
    i = index.bisect_left(value_att)
    cle = index.keys()[i]
    page = index[cle]
    if cle != value_att :
        page -=1 

    #suppression de l'ancienne entrée d'index 
    with open(index_dir + f"/page{page}", "r+") as f:
        lines = f.readlines()
        for i , line in enumerate(lines):
            cle , num_page2 , position2 = line.strip().split(',')
            if int(cle) == value_att and num_page2==num_page and position2==position:
                lines.remove(i)
                break
        f.seek(0)
        f.writelines(''.join(lines))  

    #insertion de la nouvelle entrée d'index
    i = index.bisect_left(new_value_att)
    cle = index.keys()[i]
    page = index[cle] - 1
    with open(index_dir + f"/page{page}", "r+") as f:
        lines = f.readlines()
        new_lines = []
        for i , line in enumerate(lines):
            cle , num_page2 , position2 = line.strip().split(',')
            if int(cle) > new_value_att :
                if i!=0 :
                    new_lines = lines[:i]
                new_lines.append(str(new_value_att)+","+str(num_page)+","+str(position)+"\n")
                new_lines.extend(lines[i:])
                break
        if len(new_lines)==0:
            new_lines=lines
            new_lines.append(str(new_value_att)+","+str(num_page)+","+str(position)+"\n")
        f.seek(0)
        f.writelines(''.join(new_lines))  
   
        
def modification_bis(index1, index_non_dense_a3, table, v):
    #je pense qu'ici on doit utiliser l'index non dense de a1 (c'est ce qu'on fait dans cette partie)

    index_dir = index_non_unique_dir_name(table, 1)    
    i = index1.bisect_left(v)
    cle = index1.keys()[i]
    page = index1[cle]
    if cle != v :
        page -=1 

    #ON récupere tous les rowids des tuples à modifier qui ont la val de l'attribut a1 = v
    dict_page_pos= {}
    cle_retrouvee = False
    with open(index_dir + f"/page{page}", "r+") as f:
        lines = f.readlines()
        for i , line in enumerate(lines):
            cle , num_page2 , position2 = line.strip().split(',')
            if int(cle) == v :
                if(int(num_page2) in dict_page_pos):
                    dict_page_pos[int(num_page2)].append(int(position2))
                else :
                    dict_page_pos[int(num_page2)]=[int(position2)]
                cle_retrouvee = True
            elif cle_retrouvee :
                break

    
    page_dir = page_dir_name(table)
    for num_page in dict_page_pos.keys() :
        with open(page_dir + f"/page{num_page}", "r+") as f:
            lines = f.readlines()
            for position in dict_page_pos[num_page] :
                a3 = int(lines[position].strip().split(',')[3])
                print("Valuer initiale : ",a3)
                update_table_index(table , index_non_dense_a3 , 3 , a3 , a3 + 0.5 , num_page , position) 
                a3 += 0.5
                print("Valuer apres modification : ",a3)
                l = lines[position]
                l = l.strip().split(',')
                l[3] = " "+str(a3)
                lines[position] =','.join(l)+"\n"
            f.seek(0)
            f.writelines(''.join(lines))




In [85]:
#test :
v = 7765780
print("avant la mise à jour ", acces_par_index(INDEX_a1,TABLE,v) )
t1 = time.time()
modification_bis(index_a1_non_dense, index_a3_non_dense, TABLE, v)
t2 = round(time.time() - t1, 2)
print("aprés la mise à jour " , acces_par_index(INDEX_a1,TABLE,v) )
print("\nDone in", t2, "s")

avant la mise à jour  ['5424120, 7765780, 1292450, 1362210, 330190, 401730, 206430,----------------------------------------------------------------------------------------------------', '12563170, 7765780, 3791110, 1353460, 512600, 261000, 305670,----------------------------------------------------------------------------------------------------', '6375430, 7765780, 3043010, 1797280, 637490, 539070, 252720,----------------------------------------------------------------------------------------------------']
Valuer initiale :  1362210


Valuer apres modification :  1362210.5
Valuer initiale :  1353460
Valuer apres modification :  1353460.5
Valuer initiale :  1797280
Valuer apres modification :  1797280.5
aprés la mise à jour  ['5424120, 7765780, 1292450, 1362210.5, 330190, 401730, 206430,----------------------------------------------------------------------------------------------------', '12563170, 7765780, 3791110, 1353460.5, 512600, 261000, 305670,----------------------------------------------------------------------------------------------------', '6375430, 7765780, 3043010, 1797280.5, 637490, 539070, 252720,----------------------------------------------------------------------------------------------------']

Done in 1.3 s


# Exercice 5 : Index bitmap
*   Proposer un index ayant une structure matricielle ("bitmap") pour l'attribut $a_5$. Idem pour l'attribut $a_6$.
*   En utilisant les 2 index bitmap, rechercher les tuples de T tels que $a_5 = v_1$ et $a_6 = v_2$ pour deux valeurs $v_1, v_2$ appartenant au domaine de $a_5 \cap a_6$ .




## Facultatif: Index couvrant une requête
* Illustrer les cas vus en TD pour lesquels il est possible d'obtenir le résultat d'une requête sans lire les données de la table lais seulement les index.
